### Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import json
import re
import string
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from nltk.corpus import stopwords
import pyarabic.araby as araby
from pyarabic.number import ArNumbers
from tashaphyne.stemming import ArabicLightStemmer
import qalsadi.lemmatizer as lem
from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer

### Loading the BeautifulSoup object

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
result = requests.get('https://www.hespress.com/economie', headers=headers)
doc = BeautifulSoup(result.text, 'html.parser')

### Retrieving the divs that contain the links to articles

In [3]:
main_div = doc.find_all(name='div',attrs={'class':'posts-categoy'})
posts = main_div[0].find_all(name='div',attrs={'class':'card-img-top'})
articals = []
for article in posts:
    # links
    link = article.find(name='a').get('href')
    # article titles
    title = article.find(name='a').get('title')
    articals.append({title : link})

### Storing links and titles in a json file

In [4]:
with open('data/hess_article_links.json', 'w', encoding='utf-8') as f:
    data = []
    for article in articals:
        title = list(article.keys())[0]
        link = list(article.values())[0]
        data.append({'title': title, 'link': link})
    json.dump(data, f, ensure_ascii=False, indent=2)

### Retrieving the divs that contain the links to articles

In [5]:
with open('data/hess_article_links.json', 'r', encoding='utf-8') as f:
    article_content = []
    data = json.load(f)
    for article in data:
        article_result = requests.get(article['link'], headers=headers)
        article_doc = BeautifulSoup(article_result.text, 'html.parser')
        article_content.append(article_doc.find_all(name='div',attrs={'class':'article-content'})[0].find_all(name='p'))

### Storing articles and their respective titles in a json file

In [6]:
with open('data/hess_article_content.json', 'w', encoding='utf-8') as f:
    contents = []
    for i,content in enumerate(article_content):
        content = ''
        for p in article_content[i]:
            content += p.text
        contents.append({'title': list(data[i].values())[0], 'content': content})
    json.dump(contents, f, ensure_ascii=False, indent=2)

### Uploading the two json files in a MongoDB database

In [7]:
with open('data/hess_article_links.json', 'r', encoding='utf-8') as f1:
    links = json.load(f1)
with open('data/hess_article_content.json', 'r', encoding='utf-8') as f2:
    art_content = json.load(f2)
client = MongoClient('mongodb://localhost:27017/')
db = client['NLP_lab1']
coll1 = db['hess_article_links']
coll2 = db['hess_article_content']
coll1.drop()
coll1.insert_many(links)
coll2.drop()
coll2.insert_many(art_content)
client.close()

### Loading the first article for the NLP pipeline

In [10]:
text = art_content[0]['content']
text.encode('utf-8')
print("Raw text: ",text)

Raw text:  أنهت أسعار العقود الآجلة للذهب تعاملات اليوم الأربعاء بتراجع ملموس بعد ارتفاعها ثلاثة أيام على التوالي، حيث تضرر المعدن الأصفر من ارتفاع الدولار على خلفية ارتفاع أسعار المستهلك في الولايات المتحدة بأكثر من التوقعات، مما يقلل احتمالات خفض الفائدة الأمريكية قريبا.وبعد إعلان بيانات التضخم، اليوم، تراجعت فرص خفض الفائدة الأمريكية خلال يونيو المقبل إلى 15,1 بالمئة فقط وفقا لمؤشر مراقبة مجلس الاحتياط الاتحادي.وارتفع مؤشر قيمة الدولار أمام العملات الرئيسية إلى 105,30 نقطة، قبل أن يقلص مكاسبه ليسجل 105,20 نقطة في تعاملات الظهيرة بارتفاع قدره نقطة مئوية تقريبا عن مستواه أمس.وارتفع سعر المعدن الأصفر بمقدار 13,9 دولارات، بنسبة 0,6 بالمئة إلى 2329,60 دولارا للأوقية تسليم أبريل الحالي.كما ارتفع سعر الفضة بمقدار 0,070 دولارا، بنسبة 0,215 بالمئة إلى 25,961 دولارا للأوقية تسليم أبريل الحالي.وأظهرت بيانات وزارة العمل الأمريكية، الصادرة اليوم الأربعاء، ارتفاع مؤشر أسعار المستهلك في الولايات المتحدة خلال الشهر الماضي بأكثر من التوقعات نتيجة ارتفاع أسعار المسكن والوقود.وذكرت الوزارة أن أسعار ال

### Tokenization

In [11]:
tokens = araby.tokenize(text)
sent_tokens = araby.sentence_tokenize(text)
print("Word tokenized list: ", tokens)
print("Sentence tokenized list: ", sent_tokens)
print("Word tokenized list length: ", len(tokens))
print("Sentence tokenized list length: ", len(sent_tokens))

Word tokenized list:  ['أنهت', 'أسعار', 'العقود', 'الآجلة', 'للذهب', 'تعاملات', 'اليوم', 'الأربعاء', 'بتراجع', 'ملموس', 'بعد', 'ارتفاعها', 'ثلاثة', 'أيام', 'على', 'التوالي', '،', 'حيث', 'تضرر', 'المعدن', 'الأصفر', 'من', 'ارتفاع', 'الدولار', 'على', 'خلفية', 'ارتفاع', 'أسعار', 'المستهلك', 'في', 'الولايات', 'المتحدة', 'بأكثر', 'من', 'التوقعات', '،', 'مما', 'يقلل', 'احتمالات', 'خفض', 'الفائدة', 'الأمريكية', 'قريبا', '.', 'وبعد', 'إعلان', 'بيانات', 'التضخم', '،', 'اليوم', '،', 'تراجعت', 'فرص', 'خفض', 'الفائدة', 'الأمريكية', 'خلال', 'يونيو', 'المقبل', 'إلى', '15', ',', '1', 'بالمئة', 'فقط', 'وفقا', 'لمؤشر', 'مراقبة', 'مجلس', 'الاحتياط', 'الاتحادي', '.', 'وارتفع', 'مؤشر', 'قيمة', 'الدولار', 'أمام', 'العملات', 'الرئيسية', 'إلى', '105', ',', '30', 'نقطة', '،', 'قبل', 'أن', 'يقلص', 'مكاسبه', 'ليسجل', '105', ',', '20', 'نقطة', 'في', 'تعاملات', 'الظهيرة', 'بارتفاع', 'قدره', 'نقطة', 'مئوية', 'تقريبا', 'عن', 'مستواه', 'أمس', '.', 'وارتفع', 'سعر', 'المعدن', 'الأصفر', 'بمقدار', '13', ',', '9', 'دولارا

### Removing punctuation

In [12]:
ar_punct = ''')(+`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”،.”…“–ـ”.'''
en_punct = string.punctuation
punct_lst = ar_punct + en_punct

In [13]:
tokens = [token for token in tokens if token not in punct_lst]
sent_tokens = [sent.translate(str.maketrans('', '', punct_lst)) for sent in sent_tokens]
print("Tokenized text without punctuation: ",tokens)
print("Sentence tokenized text without punctuation: ",sent_tokens)
print("Length of tokenized text without punctuation: ", len(tokens))
print("Length of sentence tokenized text without punctuation: ", len(sent_tokens))

Tokenized text without punctuation:  ['أنهت', 'أسعار', 'العقود', 'الآجلة', 'للذهب', 'تعاملات', 'اليوم', 'الأربعاء', 'بتراجع', 'ملموس', 'بعد', 'ارتفاعها', 'ثلاثة', 'أيام', 'على', 'التوالي', 'حيث', 'تضرر', 'المعدن', 'الأصفر', 'من', 'ارتفاع', 'الدولار', 'على', 'خلفية', 'ارتفاع', 'أسعار', 'المستهلك', 'في', 'الولايات', 'المتحدة', 'بأكثر', 'من', 'التوقعات', 'مما', 'يقلل', 'احتمالات', 'خفض', 'الفائدة', 'الأمريكية', 'قريبا', 'وبعد', 'إعلان', 'بيانات', 'التضخم', 'اليوم', 'تراجعت', 'فرص', 'خفض', 'الفائدة', 'الأمريكية', 'خلال', 'يونيو', 'المقبل', 'إلى', '15', '1', 'بالمئة', 'فقط', 'وفقا', 'لمؤشر', 'مراقبة', 'مجلس', 'الاحتياط', 'الاتحادي', 'وارتفع', 'مؤشر', 'قيمة', 'الدولار', 'أمام', 'العملات', 'الرئيسية', 'إلى', '105', '30', 'نقطة', 'قبل', 'أن', 'يقلص', 'مكاسبه', 'ليسجل', '105', '20', 'نقطة', 'في', 'تعاملات', 'الظهيرة', 'بارتفاع', 'قدره', 'نقطة', 'مئوية', 'تقريبا', 'عن', 'مستواه', 'أمس', 'وارتفع', 'سعر', 'المعدن', 'الأصفر', 'بمقدار', '13', '9', 'دولارات', 'بنسبة', '0', '6', 'بالمئة', 'إلى', '2329

### Removing stopwords

In [14]:
stop_words = set(stopwords.words('arabic'))
print("Stop words: ",stop_words)
tokens = [word for word in tokens if word not in stop_words]
sent_tokens = [''.join([x for x in re.split(r'(\W+)', sent) if x not in stop_words]) for sent in sent_tokens]

print("Tokenized text without stopwords: ",tokens)
print("Sentence tokenized text without stopwords: ",sent_tokens)
print("Length of tokenized text without stopwords: ", len(tokens))
print("Length of sentence tokenized text without stopwords: ", len(sent_tokens))

Stop words:  {'آب', 'تانِ', 'بل', 'إذ', 'باء', 'ذال', 'تلكم', 'مايو', 'ي', 'ذيت', 'عدَّ', 'كلَّا', 'نفس', 'شيكل', 'فيما', 'خمس', 'لن', 'نوفمبر', 'بضع', 'عسى', 'هَذِه', 'عل', 'فمن', 'هَذانِ', 'جوان', 'حاشا', 'ة', 'كثيرا', 'هما', 'أجل', 'أنّى', 'يونيو', 'سبتمبر', 'كأنما', 'رزق', 'حدَث', 'ذِه', 'مكانَك', 'هَذِي', 'غالبا', 'ضاد', 'اثنين', 'أن', 'تفعلون', 'اللتين', 'لولا', 'ألف', 'كأيّ', 'أمامكَ', 'ثالث', 'كيت', 'فوق', 'أكتوبر', 'هاتين', 'يورو', 'تسعمئة', 'شمال', 'آناء', 'ثماني', 'أولئك', 'إيه', 'سمعا', 'عدا', 'إياهما', 'إياكم', 'ثلاثة', 'الذي', 'إي', 'كليكما', 'لي', 'تسعة', 'ولكن', 'ليسا', 'تحت', 'خمسة', 'به', 'سبعة', 'ستين', 'أ', 'هذه', 'طاء', 'تَيْنِ', 'واو', 'أفٍّ', 'صبر', 'هاتان', 'ثاء', 'سحقا', 'هَاتَيْنِ', 'مازال', 'ثاني', 'ذانك', 'كأيّن', 'أنت', 'بين', 'لكنما', 'عين', 'تلقاء', 'بَلْهَ', 'ولو', 'سبعمئة', 'قبل', 'خمسون', 'كسا', 'أبو', 'ح', 'صبرا', 'بَسْ', 'بات', 'كما', 'أربعمئة', 'ديسمبر', 'آهِ', 'أقبل', 'اخلولق', 'حيث', 'ومن', 'لا سيما', 'أمام', 'عاشر', 'بما', 'هاك', 'ص', 'ذه', 'ثمان

### Convert numbers to words

In [15]:
arnum = ArNumbers()
num_tokens = []
for i,token in enumerate(tokens):
    if tokens[i].isdigit():
        tokens[i] = arnum.int2str(tokens[i])
        num_tokens.append(tokens[i])
print("Number to string: ", tokens)
print("Numerical tokens: ", num_tokens)

Number to string:  ['أنهت', 'أسعار', 'العقود', 'الآجلة', 'للذهب', 'تعاملات', 'اليوم', 'الأربعاء', 'بتراجع', 'ملموس', 'ارتفاعها', 'أيام', 'التوالي', 'تضرر', 'المعدن', 'الأصفر', 'ارتفاع', 'الدولار', 'خلفية', 'ارتفاع', 'أسعار', 'المستهلك', 'الولايات', 'المتحدة', 'بأكثر', 'التوقعات', 'يقلل', 'احتمالات', 'خفض', 'الفائدة', 'الأمريكية', 'قريبا', 'وبعد', 'إعلان', 'بيانات', 'التضخم', 'اليوم', 'تراجعت', 'فرص', 'خفض', 'الفائدة', 'الأمريكية', 'خلال', 'المقبل', 'خمس عشرة', 'واحد', 'بالمئة', 'فقط', 'وفقا', 'لمؤشر', 'مراقبة', 'مجلس', 'الاحتياط', 'الاتحادي', 'وارتفع', 'مؤشر', 'قيمة', 'الدولار', 'العملات', 'الرئيسية', 'مئة و خمس', 'ثلاثون', 'نقطة', 'يقلص', 'مكاسبه', 'ليسجل', 'مئة و خمس', 'عشرون', 'نقطة', 'تعاملات', 'الظهيرة', 'بارتفاع', 'قدره', 'نقطة', 'مئوية', 'تقريبا', 'مستواه', 'وارتفع', 'سعر', 'المعدن', 'الأصفر', 'بمقدار', 'ثلاث عشرة', 'تسع', 'دولارات', 'بنسبة', 'صفر', 'ست', 'بالمئة', 'ألفان و ثلاثمئة و تسع و عشرون', 'ستون', 'دولارا', 'للأوقية', 'تسليم', 'الحالي', 'ارتفع', 'سعر', 'الفضة', 'بمقدار',

### Normalization

In [16]:
ArListem = ArabicLightStemmer()
# removing numerical tokens
for i in num_tokens:
    tokens.remove(i)
tokens = [ArListem.normalize(token) for token in tokens]
print("Normalized token list: ", tokens)

Normalized token list:  ['انهت', 'اسعار', 'العقود', 'الاجله', 'للذهب', 'تعاملات', 'اليوم', 'الاربعاء', 'بتراجع', 'ملموس', 'ارتفاعها', 'ايام', 'التوالي', 'تضرر', 'المعدن', 'الاصفر', 'ارتفاع', 'الدولار', 'خلفيه', 'ارتفاع', 'اسعار', 'المستهلك', 'الولايات', 'المتحده', 'باكثر', 'التوقعات', 'يقلل', 'احتمالات', 'خفض', 'الفاءده', 'الامريكيه', 'قريبا', 'وبعد', 'اعلان', 'بيانات', 'التضخم', 'اليوم', 'تراجعت', 'فرص', 'خفض', 'الفاءده', 'الامريكيه', 'خلال', 'المقبل', 'بالمءه', 'فقط', 'وفقا', 'لمءشر', 'مراقبه', 'مجلس', 'الاحتياط', 'الاتحادي', 'وارتفع', 'مءشر', 'قيمه', 'الدولار', 'العملات', 'الرءيسيه', 'نقطه', 'يقلص', 'مكاسبه', 'ليسجل', 'نقطه', 'تعاملات', 'الظهيره', 'بارتفاع', 'قدره', 'نقطه', 'مءويه', 'تقريبا', 'مستواه', 'وارتفع', 'سعر', 'المعدن', 'الاصفر', 'بمقدار', 'دولارات', 'بنسبه', 'بالمءه', 'دولارا', 'للاوقيه', 'تسليم', 'الحالي', 'ارتفع', 'سعر', 'الفضه', 'بمقدار', 'دولارا', 'بنسبه', 'بالمءه', 'دولارا', 'للاوقيه', 'تسليم', 'الحالي', 'واظهرت', 'بيانات', 'وزاره', 'العمل', 'الامريكيه', 'الصادره', 'ا

### Stemming

In [17]:
stemmed_tokens = [ArListem.light_stem(token) for token in tokens]
print("Stemmed token list: ", stemmed_tokens)

Stemmed token list:  ['نهت', 'سعار', 'عقود', 'اجله', 'ذهب', 'تعامل', 'يوم', 'اربعاء', 'تراجع', 'ملموس', 'رتفاع', 'يام', 'توال', 'ضرر', 'معدن', 'اصفر', 'رتفاع', 'دولار', 'خلف', 'رتفاع', 'سعار', 'مستهلك', 'ولا', 'متحده', 'اكثر', 'توقع', 'قلل', 'احتمال', 'خفض', 'فاءده', 'امريكيه', 'قريب', 'بعد', 'علا', 'يان', 'تضخم', 'يوم', 'تراجع', 'رص', 'خفض', 'فاءده', 'امريكيه', 'خلال', 'مقبل', 'مءه', 'قط', 'فق', 'مءشر', 'مراقب', 'مجلس', 'احتياط', 'اتحاد', 'رتفع', 'مءشر', 'قيم', 'دولار', 'عمل', 'رءيسيه', 'قط', 'قلص', 'مكاسب', 'سجل', 'قط', 'تعامل', 'ظهيره', 'ارتفاع', 'قدر', 'قط', 'مءو', 'قريب', 'مستوا', 'رتفع', 'سعر', 'معدن', 'اصفر', 'مقدار', 'دولار', 'نسب', 'مءه', 'دولار', 'اوقيه', 'سليم', 'حال', 'رتفع', 'سعر', 'فضه', 'مقدار', 'دولار', 'نسب', 'مءه', 'دولار', 'اوقيه', 'سليم', 'حال', 'اظهر', 'يان', 'زار', 'عمل', 'امريكيه', 'صادره', 'يوم', 'اربعاء', 'رتفاع', 'مءشر', 'سعار', 'مستهلك', 'ولا', 'متحده', 'خلال', 'شهر', 'ماض', 'اكثر', 'توقع', 'تيج', 'رتفاع', 'سعار', 'مسكن', 'وقود', 'ذكر', 'وزاره', 'سعار', 'مسته

### Lemmatization

In [18]:
lemmer = lem.Lemmatizer()
lemmatized_tokens = [lemmer.lemmatize(token) for token in tokens]
print("Lemmatized token list: ", lemmatized_tokens)

Lemmatized token list:  ['نهت', 'اسعار', 'عقود', 'الاجله', 'ذهب', 'تعامل', 'يوم', 'الاربعاء', 'تراجع', 'ملموس', 'ارتفاع', 'ايام', 'توالي', 'تضرر', 'معدن', 'اصفر', 'ارتفاع', 'دولار', 'خلف', 'ارتفاع', 'اسعار', 'مستهلك', 'ولاية', 'المتحده', 'باكثر', 'توقع', 'قلل', 'احتمال', 'خفض', 'الفاءده', 'الامريكيه', 'قريب', 'بعد', 'اعلان', 'بيان', 'تضخم', 'يوم', 'تراجع', 'فرص', 'خفض', 'الفاءده', 'الامريكيه', 'خلال', 'مقبل', 'بالمءه', 'فقط', 'وفق', 'لمءشر', 'مراقب', 'مجلس', 'احتياط', 'اتحاد', 'ارتفع', 'مءشر', 'قيم', 'دولار', 'عملة', 'الرءيسيه', 'نقط', 'قلص', 'مكاسب', 'سجل', 'نقط', 'تعامل', 'الظهيره', 'ارتفاع', 'قدر', 'نقط', 'مءويه', 'تقريب', 'مستوى', 'ارتفع', 'سعر', 'معدن', 'اصفر', 'مقدار', 'دولار', 'نسب', 'بالمءه', 'دولار', 'للاوقيه', 'تسليم', 'حال', 'ارتفع', 'سعر', 'الفضه', 'مقدار', 'دولار', 'نسب', 'بالمءه', 'دولار', 'للاوقيه', 'تسليم', 'حال', 'واظهرت', 'بيان', 'زار', 'عمل', 'الامريكيه', 'الصادره', 'يوم', 'الاربعاء', 'ارتفاع', 'مءشر', 'اسعار', 'مستهلك', 'ولاية', 'المتحده', 'خلال', 'شهر', 'ماضي', 'با

### Stemming vs lemmatization comparison

In [19]:
print("Original token list: ", tokens)
print("Stemmed token list: ", stemmed_tokens)
print("Lemmatized token list: ", lemmatized_tokens)

Original token list:  ['انهت', 'اسعار', 'العقود', 'الاجله', 'للذهب', 'تعاملات', 'اليوم', 'الاربعاء', 'بتراجع', 'ملموس', 'ارتفاعها', 'ايام', 'التوالي', 'تضرر', 'المعدن', 'الاصفر', 'ارتفاع', 'الدولار', 'خلفيه', 'ارتفاع', 'اسعار', 'المستهلك', 'الولايات', 'المتحده', 'باكثر', 'التوقعات', 'يقلل', 'احتمالات', 'خفض', 'الفاءده', 'الامريكيه', 'قريبا', 'وبعد', 'اعلان', 'بيانات', 'التضخم', 'اليوم', 'تراجعت', 'فرص', 'خفض', 'الفاءده', 'الامريكيه', 'خلال', 'المقبل', 'بالمءه', 'فقط', 'وفقا', 'لمءشر', 'مراقبه', 'مجلس', 'الاحتياط', 'الاتحادي', 'وارتفع', 'مءشر', 'قيمه', 'الدولار', 'العملات', 'الرءيسيه', 'نقطه', 'يقلص', 'مكاسبه', 'ليسجل', 'نقطه', 'تعاملات', 'الظهيره', 'بارتفاع', 'قدره', 'نقطه', 'مءويه', 'تقريبا', 'مستواه', 'وارتفع', 'سعر', 'المعدن', 'الاصفر', 'بمقدار', 'دولارات', 'بنسبه', 'بالمءه', 'دولارا', 'للاوقيه', 'تسليم', 'الحالي', 'ارتفع', 'سعر', 'الفضه', 'بمقدار', 'دولارا', 'بنسبه', 'بالمءه', 'دولارا', 'للاوقيه', 'تسليم', 'الحالي', 'واظهرت', 'بيانات', 'وزاره', 'العمل', 'الامريكيه', 'الصادره', 'الي

### Machine learning approach for PoS tagging

In [20]:
pos_tagger = FarasaPOSTagger()
tagged_text =  pos_tagger.tag(text) # [pos_tagger.tag(token) for token in lemmatized_tokens]
print("POS Tagged",tagged_text)

POS Tagged S/S أنه +ت/V+PRON أسعار/NOUN-FP ال+ عقود/DET+NOUN-MP ال+ آجل +ة/DET+ADJ+NSUFF-FS ل+/PREP ال+ ذهب/DET+NOUN-MS تعامل +ات/NOUN+NSUFF-FP ال+ يوم/DET+NOUN-MS ال+ أربعاء/DET+NOUN-MS ب+/PREP تراجع/NOUN-MS ملموس/ADJ-MS بعد/NOUN-MS ارتفاع/NOUN-MS +ها/PRON ثلاث +ة/NUM+NSUFF-FS أيام/NOUN-FP على/PREP ال+ توالي/DET+NOUN-MS ،/PUNC حيث/PART تضرر/NOUN-MS ال+ معدن/DET+NOUN-MS ال+ أصفر/DET+ADJ-MS من/PREP ارتفاع/NOUN-MS ال+ دولار/DET+NOUN-MS على/PREP خلفي +ة/NOUN+NSUFF-FS ارتفاع/NOUN-MS أسعار/NOUN-FP ال+ مستهلك/DET+NOUN-MS في/PREP ال+ ولاي +ات/DET+NOUN+NSUFF-FP ال+ متحد +ة/DET+ADJ+NSUFF-FP ب+/PREP أكثر/ADJ-MS من/PREP ال+ توقع +ات/DET+NOUN+NSUFF-FP ،/PUNC مما/PREP+PART يقلل/V احتمال +ات/NOUN+NSUFF-FP خفض/NOUN-MS ال+ فائد +ة/DET+NOUN+NSUFF-FS ال+ أمريكي +ة/DET+ADJ+NSUFF-FP قريب/ADJ-MS +ا/CASE ./PUNC و+/CONJ بعد/NOUN-MS إعلان/NOUN-MS بيان +ات/NOUN+NSUFF-FP ال+ تضخم/DET+NOUN-MS ،/PUNC ال+ يوم/DET+NOUN-MS ،/PUNC تراجع +ت/V+PRON فرص/NOUN-FP خفض/NOUN-MS ال+ فائد +ة/DET+NOUN+NSUFF-FS ال+ أمريكي +ة/DET

In [21]:
# free up memory
del pos_tagger

### Rule-Based approach for PoS tagging

In [24]:
def tag(token):
    rules_1_2 = [r'^\s*\u0643\u0627\u0644.*\s*$', r'^\s*\u0628\u0627\u0644.*\s*$', r'^\s*\u0641\u0627\u0644.*\s*$', 
                r'^\s*\u0648\u0627\u0644.*\s*$',r'^\s*\w*\u0627\u0626\u0647\b', r'^\s*\w*\u0627\u0626\u0643\b',
                r'^\s*\w*\u0627\u0626\u064a\b', r'^\s*\w*\u0627\u0624\u0643\b', r'^\s*\w*\u0627\u0624\u0647\b',
                r'^\s*\w*\u0627\u0621\u0643\b',r'^\s*\w*\u0627\u0621\u0647\b',r'^\s*\w*\u0647\u0645\u0627\b',
                r'^\s*\w*\u0643\u0645\u0627\b']
    rules_3_4 = [r'^\s*\u0633\u064a.*\s*$', r'^\s*\u0633\u062a.*\s*$', r'^\s*\u0633\u0646.*\s*$', r'^\s*\u0633\u0623.*\s*$',
                r'^\s*\u0633\u0627.*\s*$',r'^\s*\u0644\u0627.*\s*$',r'^\s*\u0644\u0623.*\s*$',r'^\s*\u0644\u0646.*\s*$',
                r'^\s*\u0644\u062a.*\s*$',r'^\s*\u0644\u064a.*\s*$', r'^\s*\w*\u064a\b',r'^\s*\w*\u0647\b',r'^\s*\w*\u0643\b',
                r'^\s*\w*\u0627\b',r'^\s*\w*\u0646\b',r'^\s*\w*\u0648\b']
    rules_5_6 = [r'^\s*\w{3}.*\u0649\s*$', r'^\s*\w{2}\u0648\w{1}\s*$',r'^\s*\w{2}\u0627\u0621\s*$',r'^\s*\w*\u0627\u062a\s*$']
    rule_7 = r'^\s*(\u064a|\u0646)\w*(\u0648\u0646|\u064a\u0646)\s*$'
    rules_8_9 = [r'^\s*[^(\u064a|\u0646)]\w*(\u0648\u0646|\u064a\u0646)\s*$',r'^\s*\u0645\w{1}\u0627\w{2}\s*$',
                 r'^\s*\u0645\w{2}\u064a\w{1}\s*$', r'^\s*\u0645\w{2}\u0627\w{1}\s*$', r'^\s*\u0645\w{1}\u062a\w{2}\s*$',
                 r'^\s*\u0645\u0646\w{3}\s*$',r'^\s*\u0645\w{2}\u0648\w{1}\s*$',r'^\s*\u0645\u062a\w{3}\s*$',
                 r'^\s*\u0645\w{2}(.)\1\s*$']
    rule_10 = [r'^\s*\w{2}\u0627\w{1}\u064a\w{1}\s*$',r'^s*\u0623\w{1}\u0627\w{1}\u064a\w{1}\s*$',
                 r'^\s*\u0645\w{2}\u0627\w{1}\u064a\w{1}\s*$',r'^\s*\w{1}\u0648\u0627\w{1}\u064a\w{1}\s*$']
    rule_11 = [r'^\s*\u0627\u0633\u062a\w{3}\s*$',r'^\s*\u0627\w{1}(.)\u0648\1\w{1}\s*$']
    if any(re.match(pattern, token) for pattern in rules_1_2):
        return 'NOUN'
    elif any(re.match(pattern, token) for pattern in rules_3_4):
        return 'VERB'
    elif any(re.match(pattern, token) for pattern in rules_5_6):
        return 'NOUN'
    elif re.match(rule_7, token):
        return 'VERB'
    elif any(re.match(pattern, token) for pattern in rules_8_9):
        return 'NOUN'
    elif any(re.match(pattern, token) for pattern in rule_10):
        return 'NOUN'
    elif any(re.match(pattern, token) for pattern in rule_11):
        return 'VERB'
    else:
        return 'NOUN'
    
tags = [tag(token) for token in lemmatized_tokens]
tags = [tagged + ' ' + token for tagged, token in zip(tags, lemmatized_tokens)]
print("Custom POS Tagged: ", tags)

Custom POS Tagged:  ['NOUN نهت', 'NOUN اسعار', 'NOUN عقود', 'VERB الاجله', 'NOUN ذهب', 'NOUN تعامل', 'NOUN يوم', 'NOUN الاربعاء', 'NOUN تراجع', 'NOUN ملموس', 'NOUN ارتفاع', 'NOUN ايام', 'VERB توالي', 'NOUN تضرر', 'VERB معدن', 'NOUN اصفر', 'NOUN ارتفاع', 'NOUN دولار', 'NOUN خلف', 'NOUN ارتفاع', 'NOUN اسعار', 'VERB مستهلك', 'NOUN ولاية', 'VERB المتحده', 'NOUN باكثر', 'NOUN توقع', 'NOUN قلل', 'NOUN احتمال', 'NOUN خفض', 'VERB الفاءده', 'VERB الامريكيه', 'NOUN قريب', 'NOUN بعد', 'VERB اعلان', 'VERB بيان', 'NOUN تضخم', 'NOUN يوم', 'NOUN تراجع', 'NOUN فرص', 'NOUN خفض', 'VERB الفاءده', 'VERB الامريكيه', 'NOUN خلال', 'NOUN مقبل', 'NOUN بالمءه', 'NOUN فقط', 'NOUN وفق', 'NOUN لمءشر', 'NOUN مراقب', 'NOUN مجلس', 'NOUN احتياط', 'NOUN اتحاد', 'NOUN ارتفع', 'NOUN مءشر', 'NOUN قيم', 'NOUN دولار', 'NOUN عملة', 'VERB الرءيسيه', 'NOUN نقط', 'NOUN قلص', 'NOUN مكاسب', 'NOUN سجل', 'NOUN نقط', 'NOUN تعامل', 'VERB الظهيره', 'NOUN ارتفاع', 'NOUN قدر', 'NOUN نقط', 'VERB مءويه', 'NOUN تقريب', 'NOUN مستوى', 'NOUN 

### Named Entity Recognition

In [25]:
ner = FarasaNamedEntityRecognizer()
ner_tags = ner.recognize(text)
print("NER Tagged text: ",ner_tags)

NER Tagged text:  أنهت/O أسعار/O العقود/O الآجلة/O للذهب/O تعاملات/O اليوم/O الأربعاء/O بتراجع/O ملموس/O بعد/O ارتفاعها/O ثلاثة/O أيام/O على/O التوالي/O ،/O حيث/O تضرر/O المعدن/O الأصفر/O من/O ارتفاع/O الدولار/O على/O خلفية/O ارتفاع/O أسعار/O المستهلك/O في/O الولايات/B-LOC المتحدة/I-LOC بأكثر/O من/O التوقعات/O ،/O مما/O يقلل/O احتمالات/O خفض/O الفائدة/O الأمريكية/O قريبا/O ./O وبعد/O إعلان/O بيانات/O التضخم/O ،/O اليوم/O ،/O تراجعت/O فرص/O خفض/O الفائدة/O الأمريكية/O خلال/O يونيو/O المقبل/O إلى/O 15,1/O بالمئة/O فقط/O وفقا/O لمؤشر/O مراقبة/O مجلس/O الاحتياط/O الاتحادي/O ./O وارتفع/O مؤشر/O قيمة/O الدولار/O أمام/O العملات/O الرئيسية/O إلى/O 105,30/O نقطة/O ،/O قبل/O أن/O يقلص/O مكاسبه/O ليسجل/O 105,20/O نقطة/O في/O تعاملات/O الظهيرة/O بارتفاع/O قدره/O نقطة/O مئوية/O تقريبا/O عن/O مستواه/O أمس/O ./O وارتفع/O سعر/O المعدن/O الأصفر/O بمقدار/O 13,9/O دولارات/O ،/O بنسبة/O 0,6/O بالمئة/O إلى/O 2329,60/O دولارا/O للأوقية/O تسليم/O أبريل/O الحالي/O ./O كما/O ارتفع/O سعر/O الفضة/O بمقدار/O 0,07

## References
- Zerrouki, T., (2023). PyArabic: A Python package for Arabic text. Journal of Open Source Software, 8(84), 4886, https://doi.org/10.21105/joss.04886
- Alkhatib, R. M., Zerrouki, T., Shquier, M. M. A., & Balla, A. (2023). Tashaphyne0.4: A new arabic light stemmer based on rhyzome modeling approach. Information Retrieval Journa, 26(14). doi: https://doi.org/10.1007/s10791-023-09429-y
- T. Zerrouki, Qalsadi, Arabic mophological analyzer Library for python.,  https://pypi.python.org/pypi/qalsadi/
- MagedSaeed. (n.d.). GitHub - MagedSaeed/farasapy: A Python implementation of Farasa toolkit. GitHub. https://github.com/MagedSaeed/farasapy?tab=readme-ov-file#want-to-cite
- Hegazi, M. O., Al-Dossari, Y., Al-Yahy, A., Al-Sumari, A., & Hilal, A. (2021). Preprocessing Arabic text on social media. Heliyon, 7(2), e06191. https://doi.org/10.1016/j.heliyon.2021.e06191 
- Sawalha, M., Atwell, E., & Abushariah, M. A. M. (2013). SALMA: Standard Arabic Language Morphological Analysis. 2013 1st International Conference on Communications, Signal Processing, and Their Applications (ICCSPA). https://doi.org/10.1109/iccspa.2013.6487311
- Hjouj, M., Alarabeyyat, A., & Olab, I. (2016). Rule based approach for Arabic part of speech tagging and name entity recognition. International Journal of Advanced Computer Science and Applications, 7(6). https://doi.org/10.14569/ijacsa.2016.070642